In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/


Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 22:56 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 22:47 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 22:56 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 22:47 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal data.tsv  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/data.tsv': File exists


In [4]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 33.237 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 1.837 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 3.197 seconds


In [7]:
%%hive
DROP TABLE IF EXISTS cant_elems;
CREATE TABLE cant_elems AS
SELECT c1, SIZE(c2), SIZE(c3)
FROM t0;

DROP TABLE IF EXISTS cant_elems;
OK
Time taken: 0.066 seconds
CREATE TABLE cant_elems AS
SELECT c1, SIZE(c2), SIZE(c3)
FROM t0;
Query ID = root_20200218230338_26ba3eef-5570-4d8b-a0a5-f0a7d64606c0
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0034, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0034/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0034
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 23:04:01,137 Stage-1 map = 0%,  reduce = 0%
2020-02-18 23:04:13,876 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 6.2 sec
MapReduce Total cumulative CPU time: 6 seconds 200 msec
Ended Job = job_1582053079980_0034
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/hive/wa

In [8]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM cant_elems;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM cant_elems;
Query ID = root_20200218230417_2e589c5f-e1a7-47ca-a5b6-69ffc2223415
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0035, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0035/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0035
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 23:04:36,849 Stage-1 map = 0%,  reduce = 0%
2020-02-18 23:04:49,679 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.74 sec
MapReduce Total cumulative CPU time: 3 seconds 740 msec
Ended Job = job_1582053079980_0035
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_h

In [9]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [10]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [11]:
!cat output/000000_0

E,3,5
A,3,4
B,4,4
A,2,4
C,4,4
A,2,5
A,3,6
B,2,3
E,4,6
B,4,6
C,4,5
C,4,3
D,4,5
E,2,3
B,2,5
D,2,4
E,3,6
D,2,3
E,4,3
E,2,3
E,2,3
E,3,3
D,3,3
A,3,5
E,2,6
E,3,6
A,3,3
E,3,5
A,2,5
C,4,6
A,2,5
D,2,6
E,2,4
B,3,6
B,3,5
D,2,3
B,2,5
C,4,3
E,2,3
E,3,3
